In [1]:
import requests

In [2]:
url_1 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2020.zip"
url_2 = "https://download.inep.gov.br/microdados/microdados_censo_da_educacao_superior_2019.zip"


In [3]:
dado_1 = requests.get(url_1, verify=False)
dado_2 = requests.get(url_2, verify=False)


/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/urllib3/connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'download.inep.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [4]:
import zipfile

with open('dado_1.zip', 'wb') as f:
	f.write(dado_1.content)
	
with open('dado_2.zip', 'wb') as f:
	f.write(dado_2.content)
	
with zipfile.ZipFile('dado_1.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")
	
with zipfile.ZipFile('dado_2.zip', 'r') as zip_ref:
	zip_ref.extractall("./data")



In [83]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col, lower

spark = SparkSession \
        .builder \
        .config("spark.sql.repl.eagerEval.enabled", "True") \
        .config("spark.sql.repl.eagerEval.maxNumRows", "100") \
        .getOrCreate()


In [6]:
df_1 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2020/dados/MICRODADOS_CADASTRO_IES_2020.CSV", sep=';', header=True, inferSchema=True)

df_2 = spark.read.csv("data/Microdados do Censo da Educaç╞o Superior 2019/dados/MICRODADOS_CADASTRO_IES_2019.CSV", sep=';', header=True, inferSchema=True)


In [7]:
for col in df_1.columns:
    df_1 = df_1.withColumnRenamed(col, col.lower())
for col in df_2.columns:
    df_2 = df_2.withColumnRenamed(col, col.lower())


In [8]:
df_final = df_1.union(df_2)

In [9]:
df_final.write.parquet("MICRODADOS_IES_CONSULTA2.parquet")

In [10]:
df_final1 = spark.read.parquet("MICRODADOS_IES_CONSULTA2.parquet")

In [11]:
df_final1.printSchema()

root
 |-- nu_ano_censo: integer (nullable = true)
 |-- no_regiao_ies: string (nullable = true)
 |-- co_regiao_ies: integer (nullable = true)
 |-- no_uf_ies: string (nullable = true)
 |-- sg_uf_ies: string (nullable = true)
 |-- co_uf_ies: integer (nullable = true)
 |-- no_municipio_ies: string (nullable = true)
 |-- co_municipio_ies: integer (nullable = true)
 |-- in_capital_ies: integer (nullable = true)
 |-- no_mesorregiao_ies: string (nullable = true)
 |-- co_mesorregiao_ies: integer (nullable = true)
 |-- no_microrregiao_ies: string (nullable = true)
 |-- co_microrregiao_ies: integer (nullable = true)
 |-- tp_organizacao_academica: integer (nullable = true)
 |-- tp_categoria_administrativa: integer (nullable = true)
 |-- no_mantenedora: string (nullable = true)
 |-- co_mantenedora: integer (nullable = true)
 |-- co_ies: integer (nullable = true)
 |-- no_ies: string (nullable = true)
 |-- sg_ies: string (nullable = true)
 |-- ds_endereco_ies: string (nullable = true)
 |-- ds_numero_

In [12]:
df_final1.createOrReplaceTempView("parquet_df_final")

### CONSULTAS REFERENTE A PESQUISA:
# Os impactos da pandemia no Ensino Superior do Brasil
## Análise comparativa do Ensino Superior no Brasil nos anos de 2019 e 2020
#

### 1. Quantas bibliotecas passaram a oferecer serviços pela internet entre os anos de 2019 e 2020?

In [108]:
spark.sql("select  nu_ano_censo as Ano_pesquisa, count(in_servico_internet) as Qtd_bibliot_ofertam_servicos_pela_internet \
from parquet_df_final \
where in_servico_internet = 1 and (nu_ano_censo = 2019 or nu_ano_censo = 2020) \
group by nu_ano_censo order by nu_ano_censo")

Ano_pesquisa,Qtd_bibliot_ofertam_servicos_pela_internet
2019,2460
2020,2339


### 2. Houve crescimento ou queda no número de instituições? Qual estado teve o maior crescimento?

In [107]:
spark.sql("SELECT t.Estado, t.Num_instituicoes_2019, t.Num_instituicoes_2020, t.Num_instituicoes_2020 - t.Num_instituicoes_2019 AS Diferenca \
           FROM ( \
               SELECT sg_uf_ies AS Estado, \
                   SUM(CASE WHEN nu_ano_censo = 2019 THEN 1 ELSE 0 END) AS Num_instituicoes_2019, \
                   SUM(CASE WHEN nu_ano_censo = 2020 THEN 1 ELSE 0 END) AS Num_instituicoes_2020 \
               FROM parquet_df_final \
               GROUP BY sg_uf_ies \
           ) t \
           order by diferenca desc")

Estado,Num_instituicoes_2019,Num_instituicoes_2020,Diferenca
SC,100,104,4
AM,23,25,2
AP,14,15,1
RR,8,8,0
MS,35,34,-1
AL,31,30,-1
RN,29,28,-1
TO,26,24,-2
PA,73,71,-2
SE,20,18,-2


### 3. Houve quantas contratações durante o período da pandemia? O número de contratações superou 2019?

In [58]:
spark.sql("select sum(qt_doc_total) as Contrato_docentes, nu_ano_censo as Ano_pesquisa from parquet_df_final group by nu_ano_censo")

Contrato_docentes,Ano_pesquisa
399428,2019
378492,2020


### 4. Quais tipos de organização acadêmica de cada região foram mais prejudicadas com demissões de funcionários em 2020? Informe o tipo de categoria administrativa

In [105]:
spark.sql("SELECT t.tp_organizacao_academica as Tipo_org_academica, t.no_regiao_ies as Regiao, t.Contrato_docentes_2019, t.Contrato_docentes_2020, t.Contrato_docentes_2020 - t.Contrato_docentes_2019 AS Diferenca \
           FROM ( \
               SELECT tp_organizacao_academica, no_regiao_ies, \
                   SUM(CASE WHEN nu_ano_censo = 2019 THEN qt_doc_total ELSE 0 END) AS Contrato_docentes_2019, \
                   SUM(CASE WHEN nu_ano_censo = 2020 THEN qt_doc_total ELSE 0 END) AS Contrato_docentes_2020 \
               FROM parquet_df_final \
               WHERE nu_ano_censo = 2019 OR nu_ano_censo = 2020 \
               GROUP BY tp_organizacao_academica, no_regiao_ies \
               ORDER BY no_regiao_ies, tp_organizacao_academica \
           ) t")


Tipo_org_academica,Regiao,Contrato_docentes_2019,Contrato_docentes_2020,Diferenca
1,Centro-Oeste,18796,18227,-569
2,Centro-Oeste,6006,6344,338
3,Centro-Oeste,10565,8452,-2113
4,Centro-Oeste,3563,3325,-238
1,Nordeste,47586,45085,-2501
2,Nordeste,13558,14011,453
3,Nordeste,22568,20385,-2183
4,Nordeste,7808,6880,-928
1,Norte,14773,13070,-1703
2,Norte,3100,3581,481


#### NUM DO TIPO DE ORG. ACADÊMICA
##### 1 - Universidade
##### 2 - Centro Universitário
##### 3 - Faculdade
##### 4 - Instituto Federal de Educação, Ciência e Tecnologia
##### 5 - Centro Federal de Educação Tecnológica

### 5. Qual estado teve o maior número de admissões de docentes entre 35 e 54 anos, em 2020.

In [106]:
spark.sql("SELECT t.sg_uf_ies as Estado, t.Docentes_35_54_2019, t.Docentes_35_54_2020, t.Docentes_35_54_2020 - t.Docentes_35_54_2019 AS Diferenca \
           FROM ( \
               SELECT sg_uf_ies, \
                   SUM(CASE WHEN nu_ano_censo = 2019 THEN qt_doc_ex_35_39 + qt_doc_ex_40_44 + qt_doc_ex_45_49 + qt_doc_ex_50_54 ELSE 0 END) AS Docentes_35_54_2019, \
                   SUM(CASE WHEN nu_ano_censo = 2020 THEN qt_doc_ex_35_39 + qt_doc_ex_40_44 + qt_doc_ex_45_49 + qt_doc_ex_50_54 ELSE 0 END) AS Docentes_35_54_2020 \
               FROM parquet_df_final \
               WHERE nu_ano_censo = 2019 OR nu_ano_censo = 2020 \
               GROUP BY sg_uf_ies \
           ) t \
           ORDER BY diferenca DESC \
           LIMIT 54")

Estado,Docentes_35_54_2019,Docentes_35_54_2020,Diferenca
SC,11136,11274,138
PB,5822,5936,114
SE,2451,2488,37
AM,3117,3135,18
RN,3889,3903,14
CE,7967,7970,3
PE,8316,8308,-8
AP,1031,1017,-14
RR,782,754,-28
MS,3649,3601,-48
